In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
df_train = pd.read_pickle('/Users/wangshuo/Downloads/AML/data/train_one_hot_100000.pkl')
df_valid = pd.read_pickle('/Users/wangshuo/Downloads/AML/data/valid_one_hot_10000.pkl')

In [17]:
df_train_label = df_train['outcome'].copy()
df_train_input = df_train.drop(columns=['people_id','activity_id','outcome'])

df_valid_label = df_valid['outcome'].copy()
df_valid_input = df_valid.drop(columns=['people_id','activity_id','outcome'])

## Dimension Reduction

### Method 1 - Remove columns with less number of '1':  Choose 309 features

In [25]:
# dimension reduction
# the dimensions that only contain small number of '1' are dropped

threshold = 0.001 # 0.1%

keys = list(df_train_input)
keys_dropped = []

for key in keys:
  tmp_sum = df_train_input[key].sum()
  tmp_ratio = tmp_sum/len(df_train_input)
#   print(key,tmp_ratio)
  
  if tmp_ratio < threshold:
    keys_dropped.append(key)
    
df_train_input_reduced = df_train_input.drop(columns=keys_dropped)
df_valid_input_reduced = df_valid_input.drop(columns=keys_dropped)

print('The columns that contain less than ' , threshold*100 ,'% of 1 are removed.')
print(len(keys_dropped),'columns are removed from original',len(keys),'columns.')

The columns that contain less than  0.1 % of 1 are removed.
18253 columns are removed from original 18562 columns.


## Logistic Regression

###  For Method 1

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=0, solver='sag',verbose=True)
model.fit(np.array(df_train_input_reduced.values),np.array(df_train_label.values))

max_iter reached after 30 seconds


/Users/wangshuo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.2s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='sag', tol=0.0001,
          verbose=True, warm_start=False)

## Test

### Score for Method 1

In [30]:
train_predict = model.predict(np.array(df_train_input_reduced.values))
train_score = accuracy_score(train_predict,np.array(df_train_label.values))
print('train score is ',train_score)

valid_predict = model.predict(np.array(df_valid_input_reduced.values))
valid_score = accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score is ',valid_score)

train score is  0.86507
valid score is  0.8678


## Dimension Reduction

### Method 2 - PCA: Choose 300 features

In [19]:
from sklearn.decomposition import PCA
estimator = PCA(n_components = 300)

pca_df_train_input = estimator.fit_transform(df_train_input)
pca_df_valid_input = estimator.transform(df_valid_input)

In [20]:
pca_df_train_input.shape
# pca_df_valid_input.shape

(100000, 300)

## Logistic Regression

###  For Method 2

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(random_state=0, solver='sag',verbose=True)
model.fit(np.array(pca_df_train_input),np.array(df_train_label.values))

max_iter reached after 28 seconds


/Users/wangshuo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.2s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='sag', tol=0.0001,
          verbose=True, warm_start=False)

## Test

### Score for Method 2

In [38]:
train_predict = model.predict(np.array(pca_df_train_input))
train_score = accuracy_score(train_predict,np.array(df_train_label.values))
print('train score is ',train_score)

valid_predict = model.predict(np.array(pca_df_valid_input))
valid_score = accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score is ',valid_score)

train score is  0.8656
valid score is  0.8677
